In [ ]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from  sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
df=pd.read_json(r"Sarcasm_Headlines_Dataset.json",lines=True)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
df.isnull().sum()

article_link    0
headline        0
is_sarcastic    0
dtype: int64

In [ ]:
df.drop('is_sarcastic', axis=1)

,article_link,headline
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up..."
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...
...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...


In [ ]:
y=df['is_sarcastic']

In [ ]:
y.value_counts()

0    14985
1    11724
Name: is_sarcastic, dtype: int64

In [ ]:
voc_size = 5000


In [ ]:
import tensorflow as tf

In [ ]:
tokens=[]
stop=stopwords.words('english')

In [ ]:

for t in df.headline:
  wt=wordpunct_tokenize(t)
  ps=PorterStemmer()
  stem=[ps.stem(w) for w in wt if w not in stop ]
  stem=" ".join(stem)
  tokens.append(stem)

#print(tokens)

cv=CountVectorizer(ngram_range=(1,2))
cv.fit(tokens)
cv.vocabulary_

{'former': 51013,
 'versac': 141326,
 'store': 126344,
 'clerk': 25351,
 'sue': 127938,
 'secret': 116518,
 'black': 15168,
 'code': 26193,
 'minor': 84381,
 'shopper': 119394,
 'former versac': 51102,
 'versac store': 141327,
 'store clerk': 126352,
 'clerk sue': 25363,
 'sue secret': 127962,
 'secret black': 116530,
 'black code': 15189,
 'code minor': 26201,
 'minor shopper': 84400,
 'roseann': 112887,
 'reviv': 111249,
 'catch': 21629,
 'thorni': 133021,
 'polit': 100473,
 'mood': 85813,
 'better': 14008,
 'wors': 148195,
 'roseann reviv': 112889,
 'reviv catch': 111251,
 'catch thorni': 21655,
 'thorni polit': 133024,
 'polit mood': 100531,
 'mood better': 85814,
 'better wors': 14102,
 'mom': 85226,
 'start': 124985,
 'fear': 47279,
 'son': 122627,
 'web': 144690,
 'seri': 117807,
 'closest': 25868,
 'thing': 132516,
 'grandchild': 57071,
 'mom start': 85375,
 'start fear': 125024,
 'fear son': 47331,
 'son web': 122738,
 'web seri': 144696,
 'seri closest': 117815,
 'closest thi

In [ ]:
cv=CountVectorizer(ngram_range=(1,1))
cv_matrix=cv.fit_transform(df.headline)
cv_matrix=cv_matrix.toarray()
cv_matrix
cv_matrix.shape

(26709, 25292)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(cv_matrix,df.is_sarcastic,test_size=0.3)

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((18696, 25292), (8013, 25292), (18696,), (8013,))

In [ ]:
Y_test.value_counts()

0    4453
1    3560
Name: is_sarcastic, dtype: int64

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model=MultinomialNB()
model.fit(X_train,Y_train)

y_pred=model.predict(X_test)
print(len(Y_test), len(y_pred))
from sklearn.metrics import classification_report,confusion_matrix

cm=confusion_matrix(Y_test,y_pred)

cr=classification_report(Y_test,y_pred)
print(cr)

8013 8013
              precision    recall  f1-score   support

           0       0.84      0.90      0.87      4453
           1       0.86      0.79      0.82      3560

    accuracy                           0.85      8013
   macro avg       0.85      0.84      0.84      8013
weighted avg       0.85      0.85      0.85      8013



In [ ]:

from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(df.headline,df.is_sarcastic,test_size=0.2,random_state=2022, stratify =df.is_sarcastic)

In [ ]:
clf = Pipeline([
    ('vectorizer_trigrams', CountVectorizer(ngram_range = (1, 1))),                   #using the ngram_range parameter
     ('KNN', (KNeighborsClassifier(n_neighbors=10, metric = 'euclidean')))           #using the KNN classifier with 10 neighbors and euclidean distance
])

#2. fit with X_train and y_train
clf.fit(X_train, Y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.97      0.73      2997
           1       0.80      0.13      0.22      2345

    accuracy                           0.60      5342
   macro avg       0.69      0.55      0.48      5342
weighted avg       0.68      0.60      0.51      5342

